In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-paper')

import warnings
warnings.filterwarnings('ignore')

<img src="logic_tree_fun.png" width="500">

## __1. 데이터 준비__

### _1.1 File Info_
- 문서 앱내 사용시 발생하는 클라이언 로그 데이터 (after parsed)
- 서버로그와 달리, 유저의 행동이 발생할 경우 로그 수집 (유저 행동 패턴 파악에 용이)
    - 클라이언트(앱) 로그
    - 서버 로그

In [3]:
df = pd.read_csv('df_funnel.csv', index_col=0)
df.tail(3)

,actiontype,ismydoc,ext,sessionid,documentposition,datetime,screen
301858,RESET,View,DOCX,41b8c1df291edd2aec30fc610e5b676e,OTHERAPP,2016.7.5,Main
301859,OPEN,NoView,PPT,9b5ee236571c2fcbe25132b6ba71d764,OTHERAPP,2016.7.14,Main
301860,CLOSE,NoView,PPT,9b5ee236571c2fcbe25132b6ba71d764,OTHERAPP,2016.7.14,Main


### _1.2 Columns Info_
- actiontype: 문서 이용시 행동(OPEN, CLOSE, SAVE,,)
- ismydoc: 내문서 해당 여부(중요x)
- ext: 문서 확장자
- sessionid: 유저 식별자
- documentposition: 문서 이용시 위치 정보(CLOUD, OTHERAPP)
- datetime: Timestamp
- screen: 앱내 화면 이름

In [7]:
display(df.actiontype.unique())
display(df.ismydoc.unique())
display(df.ext.unique())
display(df.documentposition.unique()) # NONE 데이터 존재
display(df.screen.unique())

array(['OPEN', 'CLOSE', 'RESET', 'SAVEAS', 'SAVE', 'SAVEAS_OTHER',
       'EXPORT_SAME', 'EXPORT'], dtype=object)

array(['NoView', 'View'], dtype=object)

array(['PDF', 'HWP', 'XLSX', 'PPT', 'DOCX', 'ODT', 'DOC', 'XLS', 'PPTX',
       'TXT', 'PPS', 'PPSX', 'WORD', 'PNG', 'JPG', 'SHEET'], dtype=object)

array(['LOCALSTORAGE', 'MYPOLARISDRIVE', 'OTHERAPP', 'NEWDOCUMENT',
       'NONE', 'OTHERCLOUD', 'SHAREDDOCUMENT'], dtype=object)

array(['Per_Dir', 'Pub_Dir', 'Main', 'InProduct_Web', 'InProduct_Mob',
       'Purchase_page', 'Purchase_done'], dtype=object)

## 2. 전처리

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301861 entries, 0 to 301860
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   actiontype        301861 non-null  object
 1   ismydoc           301861 non-null  object
 2   ext               301861 non-null  object
 3   sessionid         301861 non-null  object
 4   documentposition  301861 non-null  object
 5   datetime          301861 non-null  object
 6   screen            301861 non-null  object
dtypes: object(7)
memory usage: 18.4+ MB


In [20]:
df.datetime = df.datetime.astype('datetime64[ns]')
df.tail()

,actiontype,ismydoc,ext,sessionid,documentposition,datetime,screen
301856,CLOSE,View,XLSX,2ed068d5e6a72e80e4a997c01c59f782,OTHERAPP,2016-07-07,Main
301857,OPEN,NoView,DOCX,41b8c1df291edd2aec30fc610e5b676e,OTHERAPP,2016-07-05,Main
301858,RESET,View,DOCX,41b8c1df291edd2aec30fc610e5b676e,OTHERAPP,2016-07-05,Main
301859,OPEN,NoView,PPT,9b5ee236571c2fcbe25132b6ba71d764,OTHERAPP,2016-07-14,Main
301860,CLOSE,NoView,PPT,9b5ee236571c2fcbe25132b6ba71d764,OTHERAPP,2016-07-14,Main


In [23]:
display(df.datetime.dt.year[:5])
display(df.datetime.dt.month[:5])
display(df.datetime.dt.day[:5])

0    2016
1    2016
2    2016
3    2016
4    2016
Name: datetime, dtype: int64

0    7
1    7
2    7
3    7
4    7
Name: datetime, dtype: int64

0    18
1    18
2    18
3    18
4     6
Name: datetime, dtype: int64

### _2.1 결측치 처리_

In [24]:
df.isnull().sum()

actiontype          0
ismydoc             0
ext                 0
sessionid           0
documentposition    0
datetime            0
screen              0
dtype: int64

In [11]:
df_by_screen = df.groupby(['datetime','screen'])['sessionid'].nunique().unstack()
df_by_screen

screen,InProduct_Mob,InProduct_Web,Main,Per_Dir,Pub_Dir,Purchase_done,Purchase_page
datetime,,,,,,,
2016.7.1,137.0,147.0,3582.0,296.0,1008.0,2.0,23.0
2016.7.10,62.0,122.0,1872.0,230.0,636.0,2.0,18.0
2016.7.11,125.0,139.0,2944.0,332.0,801.0,NaN,26.0
2016.7.12,138.0,157.0,3646.0,371.0,1000.0,4.0,26.0
2016.7.13,132.0,172.0,3544.0,334.0,988.0,1.0,35.0
2016.7.14,131.0,153.0,3479.0,364.0,923.0,5.0,16.0
2016.7.15,95.0,147.0,3281.0,320.0,875.0,2.0,22.0
2016.7.16,91.0,145.0,2483.0,248.0,716.0,1.0,22.0
2016.7.17,75.0,134.0,1903.0,277.0,696.0,3.0,20.0


In [26]:
df_by_screen.isnull().sum()

screen
InProduct_Mob    0
InProduct_Web    0
Main             0
Per_Dir          0
Pub_Dir          0
Purchase_done    6
Purchase_page    0
dtype: int64

In [30]:
df_by_screen.fillna(0, inplace=True)
df_by_screen

screen,InProduct_Mob,InProduct_Web,Main,Per_Dir,Pub_Dir,Purchase_done,Purchase_page
datetime,,,,,,,
2016-07-01,137.0,147.0,3582.0,296.0,1008.0,2.0,23.0
2016-07-02,89.0,124.0,2683.0,206.0,801.0,2.0,15.0
2016-07-03,50.0,83.0,1886.0,165.0,590.0,0.0,28.0
2016-07-04,112.0,104.0,2808.0,281.0,764.0,0.0,29.0
2016-07-05,118.0,138.0,3335.0,300.0,842.0,1.0,26.0
2016-07-06,138.0,145.0,3256.0,293.0,855.0,1.0,30.0
2016-07-07,110.0,117.0,3150.0,266.0,807.0,0.0,18.0
2016-07-08,117.0,148.0,3208.0,273.0,865.0,2.0,20.0
2016-07-09,102.0,138.0,2381.0,229.0,748.0,2.0,23.0


In [58]:
x = df[['actiontype','ismydoc','ext','screen']]
y = df[['documentposition']]

In [59]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

X = x.apply(lambda x : encoder.fit_transform(x))
y = y.apply(lambda y : encoder.fit_transform(y))

In [60]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=10) # test_size = 0.3

In [61]:
x.shape, y.shape, x_train.shape, x_test.shape

((301861, 4), (301861, 1), (226395, 4), (75466, 4))

In [62]:
df.ext.unique()

array(['PDF', 'HWP', 'XLSX', 'PPT', 'DOCX', 'ODT', 'DOC', 'XLS', 'PPTX',
       'TXT', 'PPS', 'PPSX', 'WORD', 'PNG', 'JPG', 'SHEET'], dtype=object)

In [96]:
ext_change = {'XLSX':'XLS', 'DOCX':'DOC', 'PPTX':'PPT', 'PPS':'PPT', 'PPSX':'PPT', 'ODT':'TXT', 'PNG':'JPG'}

df.ext.replace(ext_change, inplace=True)
df.ext.unique()

array(['PDF', 'HWP', 'XLS', 'PPT', 'DOC', 'TXT', 'WORD', 'JPG', 'SHEET'],
      dtype=object)

In [77]:
result = []
j = 0
for i in range(len(df)-1):
    if df.sessionid[i] == df.sessionid[i+1]:
        result.append(j)
        if i == (len(df)-2):
            result.append(j)
    else:
        result.append(j)
        j += 1
        if i == (len(df)-2):
            result.append(j)

In [78]:
result[-10:]

[132149,
 132149,
 132150,
 132150,
 132151,
 132151,
 132152,
 132152,
 132153,
 132153]

In [97]:
df['id'] = pd.Series(result)
df.tail()

,actiontype,ismydoc,ext,sessionid,documentposition,datetime,screen,id
301856,CLOSE,View,XLS,2ed068d5e6a72e80e4a997c01c59f782,OTHERAPP,2016.7.7,Main,132151
301857,OPEN,NoView,DOC,41b8c1df291edd2aec30fc610e5b676e,OTHERAPP,2016.7.5,Main,132152
301858,RESET,View,DOC,41b8c1df291edd2aec30fc610e5b676e,OTHERAPP,2016.7.5,Main,132152
301859,OPEN,NoView,PPT,9b5ee236571c2fcbe25132b6ba71d764,OTHERAPP,2016.7.14,Main,132153
301860,CLOSE,NoView,PPT,9b5ee236571c2fcbe25132b6ba71d764,OTHERAPP,2016.7.14,Main,132153


In [98]:
col = list(df.columns)
col.remove('datetime')
col.remove('id')
col

['actiontype', 'ismydoc', 'ext', 'sessionid', 'documentposition', 'screen']

In [99]:
df[col] = df[col].applymap(lambda x : x.upper())
df

,actiontype,ismydoc,ext,sessionid,documentposition,datetime,screen,id
0,OPEN,NOVIEW,PDF,9400FD2E43D7DC2D054CA78806236EE1,LOCALSTORAGE,2016.7.18,PER_DIR,0
1,CLOSE,NOVIEW,PDF,9400FD2E43D7DC2D054CA78806236EE1,LOCALSTORAGE,2016.7.18,PER_DIR,0
2,OPEN,VIEW,PDF,9400FD2E43D7DC2D054CA78806236EE1,MYPOLARISDRIVE,2016.7.18,PUB_DIR,0
3,CLOSE,VIEW,PDF,9400FD2E43D7DC2D054CA78806236EE1,MYPOLARISDRIVE,2016.7.18,PUB_DIR,0
4,OPEN,NOVIEW,PDF,F191063C562691041DFA935FF0876975,OTHERAPP,2016.7.6,MAIN,1
...,...,...,...,...,...,...,...,...
301856,CLOSE,VIEW,XLS,2ED068D5E6A72E80E4A997C01C59F782,OTHERAPP,2016.7.7,MAIN,132151
301857,OPEN,NOVIEW,DOC,41B8C1DF291EDD2AEC30FC610E5B676E,OTHERAPP,2016.7.5,MAIN,132152
301858,RESET,VIEW,DOC,41B8C1DF291EDD2AEC30FC610E5B676E,OTHERAPP,2016.7.5,MAIN,132152
301859,OPEN,NOVIEW,PPT,9B5EE236571C2FCBE25132B6BA71D764,OTHERAPP,2016.7.14,MAIN,132153
